In [3]:
import numpy as np
import pandas as pd
names = ["Sex", 
"Length",
"Diameter",
"Height",
"whole weight",
"Shucked weight",
"Viscera weight",
"Shell weight",
"Rings"]
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data",
                  names = names)
data.head()

,Sex,Length,Diameter,Height,whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [4]:
from collections import Counter
Counter(data["Sex"])

Counter({'M': 1528, 'F': 1307, 'I': 1342})

In [5]:
df_gender = pd.get_dummies(data['Sex'])
data = pd.concat([data, df_gender], axis=1)
data.drop(['Sex'],axis=1,inplace=True)

In [6]:
minority_class = data[data.Rings==18]

In [7]:
majority_class = data[data.Rings==9]

In [8]:
data = pd.concat([minority_class,majority_class],ignore_index=True)

In [9]:
columns=data.columns
data.head()

,Length,Diameter,Height,whole weight,Shucked weight,Viscera weight,Shell weight,Rings,F,I,M
0,0.665,0.525,0.165,1.3380,0.5515,0.3575,0.350,18,0,0,1
1,0.710,0.540,0.165,1.9590,0.7665,0.2610,0.780,18,0,0,1
2,0.725,0.560,0.210,2.1410,0.6500,0.3980,1.005,18,1,0,0
3,0.610,0.500,0.240,1.6420,0.5320,0.3345,0.690,18,0,0,1
4,0.580,0.455,0.155,0.8365,0.3150,0.1385,0.320,18,1,0,0


In [10]:
data["Rings"] = np.where(data["Rings"]==18,1,0)

In [11]:
data.head()

,Length,Diameter,Height,whole weight,Shucked weight,Viscera weight,Shell weight,Rings,F,I,M
0,0.665,0.525,0.165,1.3380,0.5515,0.3575,0.350,1,0,0,1
1,0.710,0.540,0.165,1.9590,0.7665,0.2610,0.780,1,0,0,1
2,0.725,0.560,0.210,2.1410,0.6500,0.3980,1.005,1,1,0,0
3,0.610,0.500,0.240,1.6420,0.5320,0.3345,0.690,1,0,0,1
4,0.580,0.455,0.155,0.8365,0.3150,0.1385,0.320,1,1,0,0


In [12]:
print(sum(data['Rings']==1),
sum(data['Rings']==0))

42 689


In [13]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
scaler = MinMaxScaler()
model = scaler.fit(data)
data = pd.DataFrame(model.transform(data),columns=columns)
data.head()

,Length,Diameter,Height,whole weight,Shucked weight,Viscera weight,Shell weight,Rings,F,I,M
0,0.827957,0.842105,0.666667,0.550193,0.419048,0.641251,0.321244,1.0,0.0,0.0,1.0
1,0.924731,0.881579,0.666667,0.831634,0.597101,0.452590,0.766839,1.0,0.0,0.0,1.0
2,0.956989,0.934211,0.866667,0.914117,0.500621,0.720430,1.000000,1.0,1.0,0.0,0.0
3,0.709677,0.776316,1.000000,0.687967,0.402899,0.596285,0.673575,1.0,0.0,0.0,1.0
4,0.645161,0.657895,0.622222,0.322910,0.223188,0.213099,0.290155,1.0,1.0,0.0,0.0


### DATA RESAMPLING

### GENERATING TEST DATA

In [15]:
X = data.drop(['Rings'],axis=1)
y = data.Rings

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=42)

In [17]:
print(sum(y_train==1)
,sum(y_train==0)
,sum(y_test==1)
,sum(y_test==0)
     )

36 585 6 104


In [18]:
(sum(y_train==1)+sum(y_test==1))/(sum(y_train==0)+sum(y_test==0))
     

0.06095791001451379

In [19]:
x_test.to_csv("abalone_test_data.csv",index=False)
y_test.to_csv("abalone_test_label.csv",index=False)

In [20]:
x_train.to_csv("abalone_train_data.csv",index=False)
y_train.to_csv("abalone_train_label.csv",index=False)

### GENERATING EXT_IMB DATASET(05:95)

In [18]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import ADASYN 
from imblearn.under_sampling import RandomUnderSampler

In [19]:
X, y = x_train,y_train
print('Original dataset shape %s' % Counter(y))

Original dataset shape Counter({0.0: 585, 1.0: 36})


In [20]:
no_minority, no_majority = (Counter(y)[1]+Counter(y)[0])*0.05,(Counter(y)[1]+Counter(y)[0])*0.95
ada = ADASYN(random_state=42,sampling_strategy={1:Counter(y)[1],0:741})
X_res, y_res = ada.fit_resample(X, y)
print('Resampled dataset shape After oversampling majority class %s' % Counter(y_res))

Resampled dataset shape After oversampling majority class Counter({0.0: 743, 1.0: 36})


/Applications/Utilities/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:318: UserWarning: After over-sampling, the number of samples (741) in class 0 will be larger than the number of samples in the majority class (class #0.0 -> 585)
  warnings.warn(


In [21]:
X_res.to_csv("abalone_ext_imb_data.csv",index=False)
y_res.to_csv("abalone_ext_imb_label.csv",index=False)

### GENERATING 15285_IMB DATASET(15:85)

In [22]:
X, y = x_train,y_train
print('Original dataset shape %s' % Counter(y))

Original dataset shape Counter({0.0: 585, 1.0: 36})


In [23]:
no_minority, no_majority = (Counter(y)[1]+Counter(y)[0])*0.15,(Counter(y)[1]+Counter(y)[0])*0.85
ada = ADASYN(random_state=42,sampling_strategy={0:Counter(y)[0],1:round(no_minority)})
X_res, y_res = ada.fit_resample(X, y)
print('Resampled dataset shape After oversampling minority class %s' % Counter(y_res))
rnd = RandomUnderSampler(random_state=42,sampling_strategy={0:round(no_majority),1:Counter(y_res)[1]})
X_final, y_final = rnd.fit_resample(X_res, y_res)
print('Resampled dataset shape with 15:85 ratio%s' % Counter(y_final))

Resampled dataset shape After oversampling minority class Counter({0.0: 585, 1.0: 90})
Resampled dataset shape with 15:85 ratioCounter({0.0: 528, 1.0: 90})


In [24]:
X_final.to_csv("abalone_15285_imb_data.csv",index=False)
y_final.to_csv("abalone_15285_imb_label.csv",index=False)

### GENERATING MOD_IMB DATASET(30:70)

In [25]:
X, y = x_train,y_train
print('Original dataset shape %s' % Counter(y))

Original dataset shape Counter({0.0: 585, 1.0: 36})


In [26]:
no_minority, no_majority = (Counter(y)[1]+Counter(y)[0])*0.3,(Counter(y)[1]+Counter(y)[0])*0.7
ada = ADASYN(random_state=42,sampling_strategy={0:Counter(y)[0],1:round(no_minority)})
X_res, y_res = ada.fit_resample(X, y)
print('Resampled dataset shape After oversampling minority class %s' % Counter(y_res))
rnd = RandomUnderSampler(random_state=42,sampling_strategy={0:round(no_majority),1:Counter(y_res)[1]})
X_final, y_final = rnd.fit_resample(X_res, y_res)
print('Resampled dataset shape with 30:70 ratio%s' % Counter(y_final))

Resampled dataset shape After oversampling minority class Counter({0.0: 585, 1.0: 190})
Resampled dataset shape with 30:70 ratioCounter({0.0: 435, 1.0: 190})


In [27]:
X_final.to_csv("abalone_mod_imb_data.csv",index=False)
y_final.to_csv("abalone_mod_imb_label.csv",index=False)

### GENERATING NO_IMB DATASET(50:50)

In [28]:
X, y = x_train,y_train
print('Original dataset shape %s' % Counter(y))

Original dataset shape Counter({0.0: 585, 1.0: 36})


In [29]:
ada = ADASYN(random_state=42)
X_res, y_res = ada.fit_resample(X, y)
print('Resampled dataset shape with 50:50 %s' % Counter(y_res))

Resampled dataset shape with 50:50 Counter({1.0: 587, 0.0: 585})


In [30]:
X_res.to_csv("abalone_no_imb_data.csv",index=False)
y_res.to_csv("abalone_no_imb_label.csv",index=False)